In [ ]:
import pandas as pd
import seaborn as sb
import math
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("all.csv")

df = df.sort_values(by=["client_protocol", "remote_protocol"])

df.head(20)

In [ ]:
# calculate derived data
df["init_failure_rate"] = df["init_failures"] / df["init_count"] * 100
df["method_failure_rate"] = df["method_call_failures"] / df["method_call_count"] * 100
df["non_idempotent_mismatch_rate"] = df["non_idempotent_mismatches"] / df["non_idempotent_calls"] * 100

# factor difference between simulated failure rate and observed failure rate
df["method_failure_factor"] = 100 / df["inverse_failure_probability"] / df["method_failure_rate"]

# calc axes variables
df["log_inverse_failure_probability"] = np.log10(df["inverse_failure_probability"])

df["log_inverse_failure_probability"].fillna(10, inplace=True)


In [ ]:
# client_remote
# segmenting the dataset into 4 partitions

not_faulty= "^((?!Faulty).)*$"

same_proto: pd.DataFrame = df[df["client_protocol"] == df["remote_protocol"]]
diff_proto: pd.DataFrame = df[df["client_protocol"] != df["remote_protocol"]]

ok_ok = same_proto[df["client_protocol"].str.contains(not_faulty)]
err_err = same_proto[df["client_protocol"].str.startswith("Faulty")]

ok_err = diff_proto[df["client_protocol"].str.contains(not_faulty)]
err_ok = diff_proto[df["client_protocol"].str.startswith("Faulty")]


ok_err


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
# all subplots share the same axes labels, remove the labels from all but the bottom left
for ax in axes[0, :]:
    ax.set_xlabel("")

data = [
    [ok_ok, ok_err],
    [err_ok, err_err]
]

titles = [
    ["Control", "Server-side failure"],
    ["Client-side failure", "Twin failure"]
]

# sb.color_palette("rocket")
sb.set_palette("mako", n_colors=3)

# fig.legend(handles=[], labels=[], loc="upper center", ncol=3)

for row in range(2):
    for col in range(2):

        axes[row, col].set_title(titles[row][col])

        # create vertical grid lines inside each subplot
        # axes[row, col].grid(which="both", axis="x", linestyle="--")
        axes[row, col].yaxis.grid()

        sb.swarmplot(
            data=data[row][col],
            x="log_inverse_failure_probability",
            y="init_failure_rate",
            ax=axes[row, col],
            hue="client_protocol",
            dodge=True,
            # legend=False
        )

        axes[row, col].legend().set_visible(False)

        # create a global legend for the entire figure based on the first subplot
        if row == 0 and col == 0:
            handles, labels = axes[row, col].get_legend_handles_labels()
            fig.legend(handles, labels, loc="upper center", ncol=3)


In [ ]:
fig.savefig("plot.svg")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
# all subplots share the same axes labels, remove the labels from all but the bottom left
for ax in axes[0, :]:
    ax.set_xlabel("")

data = [
    [ok_ok, ok_err],
    [err_ok, err_err]
]

titles = [
    ["Control", "Server-side failure"],
    ["Client-side failure", "Twin failure"]
]

# sb.color_palette("rocket")
sb.set_palette("mako", n_colors=3)

# fig.legend(handles=[], labels=[], loc="upper center", ncol=3)

for row in range(2):
    for col in range(2):

        axes[row, col].set_title(titles[row][col])

        # create vertical grid lines inside each subplot
        # axes[row, col].grid(which="both", axis="x", linestyle="--")
        axes[row, col].yaxis.grid()

        sb.swarmplot(
            data=data[row][col],
            x="log_inverse_failure_probability",
            y="non_idempotent_mismatch_rate",
            ax=axes[row, col],
            hue="client_protocol",
            dodge=True,
            # legend=False
        )

        axes[row, col].legend().set_visible(False)

        # create a global legend for the entire figure based on the first subplot
        if row == 0 and col == 0:
            handles, labels = axes[row, col].get_legend_handles_labels()
            fig.legend(handles, labels, loc="upper center", ncol=3)


In [ ]:
fig.savefig("idem.svg")
